# Importing Libraries

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import zipfile
import shutil
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
from keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')  
import gradio as gr


# Loading Dataset

In [3]:
import os
import zipfile
import shutil

# Paths for the dataset zip and extraction folder
dataset_zip = "PlantVillage.zip"
dataset_path = "PlantVillage"

# Unzipping the dataset
if not os.path.exists(dataset_path):
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall()

    # Check if an extra "PlantVillage/PlantVillage" folder was created
    nested_folder = os.path.join(dataset_path, "PlantVillage")

    if os.path.exists(nested_folder):  
        # Move files from nested folder to the correct location
        for item in os.listdir(nested_folder):
            shutil.move(os.path.join(nested_folder, item), dataset_path)

        # Remove the empty nested folder
        os.rmdir(nested_folder)

In [4]:
# List all classes in the dataset
classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

print("Plant Village Classes:")
for plant_class in classes:
    print(f" - {plant_class}")


Plant Village Classes:
 - Pepper__bell___Bacterial_spot
 - Pepper__bell___healthy
 - Potato___Early_blight
 - Potato___healthy
 - Potato___Late_blight
 - Tomato_Bacterial_spot
 - Tomato_Early_blight
 - Tomato_healthy
 - Tomato_Late_blight
 - Tomato_Leaf_Mold
 - Tomato_Septoria_leaf_spot
 - Tomato_Spider_mites_Two_spotted_spider_mite
 - Tomato__Target_Spot
 - Tomato__Tomato_mosaic_virus
 - Tomato__Tomato_YellowLeaf__Curl_Virus


# Data Preparation

In [7]:
file_paths = []
labels = []

for plant_class in classes:
    class_path = os.path.join(dataset_path, plant_class)
    for file in os.listdir(class_path):
        if file.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
            file_paths.append(os.path.join(class_path, file))
            labels.append(plant_class)

# Convert labels to numerical format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# One-hot encoding
labels_one_hot = to_categorical(labels_encoded, num_classes=len(classes))

# Train-test split (80% training, 20% validation)
train_files, val_files, train_labels, val_labels = train_test_split(
    file_paths, labels_one_hot, test_size=0.2, stratify=labels_encoded, random_state=42
)

print(f"Training samples: {len(train_files)}, Validation samples: {len(val_files)}")

Training samples: 16510, Validation samples: 4128


# Data Augmentation

In [19]:
# Enable Mixed Precision Training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Define Dataset Path
dataset_path = "PlantVillage"

# Get Class Names
classes = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])
num_classes = len(classes)

print(f"Found {num_classes} Classes: {classes}")

# Image Augmentation
datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=30, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True, 
    validation_split=0.2
)

# Data Generators
train_generator = datagen.flow_from_directory(
    dataset_path, target_size=(128, 128), batch_size=64, class_mode='categorical', subset='training')

val_generator = datagen.flow_from_directory(
    dataset_path, target_size=(128, 128), batch_size=64, class_mode='categorical', subset='validation')



Found 15 Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.


# Model Building

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import tensorflow as tf

# Function to Evaluate Each Model
def evaluate_model(model, val_generator, model_name):
    print(f"\n Evaluating {model_name}...")

    # Get True Labels & Predictions
    val_preds = model.predict(val_generator)
    val_preds_classes = np.argmax(val_preds, axis=1)
    val_true_classes = val_generator.classes

    # Compute Accuracy
    accuracy = np.mean(val_preds_classes == val_true_classes)
    print(f"\ {model_name} Accuracy: {accuracy:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(val_true_classes, val_preds_classes)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=val_generator.class_indices.keys(), yticklabels=val_generator.class_indices.keys())
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"{model_name} - Confusion Matrix")
    plt.show()

    # Compute AUC Score
    auc_score = roc_auc_score(tf.keras.utils.to_categorical(val_true_classes, num_classes), val_preds, multi_class='ovr')
    print(f"\n🔹 {model_name} AUC Score: {auc_score:.4f}")

    # ROC Curve for Each Class
    plt.figure(figsize=(10,6))
    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(tf.keras.utils.to_categorical(val_true_classes, num_classes)[:, i], val_preds[:, i])
        plt.plot(fpr, tpr, label=f"Class {i}: {classes[i]}")
    
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{model_name} - AUC Curve")
    plt.legend()
    plt.show()

    # Classification Report
    print(f"\n {model_name} Classification Report:")
    print(classification_report(val_true_classes, val_preds_classes, target_names=classes))


## VGG16 Model

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Load Pretrained VGG16 Model
vgg_base = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128, 3))

# Freeze all but the last 20 layers 
for layer in vgg_base.layers[:-20]:  
    layer.trainable = False  

# Build Model
vgg_model = Sequential([
    vgg_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax', dtype='float32') 
])

# 🔹 Compile Model
vgg_model.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

# Callbacks 
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

# Train 
history_vgg = vgg_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=100,  
    validation_steps=50,  
    callbacks=callbacks
)

Epoch 1/10


In [16]:
# Load Pretrained VGG16 Model
vgg_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
for layer in vgg_base.layers[:-30]:  
    layer.trainable = False  # Freeze most layers

# Build Model
vgg_model = Sequential([
    vgg_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile
vgg_model.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

# Train
history_vgg = vgg_model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[ReduceLROnPlateau()])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step
Epoch 1/10
  4/517 ━━━━━━━━━━━━━━━━━━━━ 72:06:07 506s/step - accuracy: 0.0527 - loss: 3.2507

KeyboardInterrupt: 

In [ ]:
vgg_model.summary()

In [ ]:
evaluate_model(vgg_model, val_generator, "VGG16")


## ResNet Model

In [ ]:
# Load Pretrained ResNet50 Model
resnet_base = ResNet50(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
for layer in resnet_base.layers[:-30]:  
    layer.trainable = False  

# Build Model
resnet_model = Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile
resnet_model.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

# Train
history_resnet = resnet_model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[ReduceLROnPlateau()])


In [ ]:
resnet_model.summary()

In [ ]:
evaluate_model(resnet_model, val_generator, "ResNet50")


## Inception Model

In [ ]:
# Load Pretrained InceptionV3 Model
inception_base = InceptionV3(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
for layer in inception_base.layers[:-30]:  
    layer.trainable = False  

# Build Model
inception_model = Sequential([
    inception_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile
inception_model.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

# Train
history_inception = inception_model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[ReduceLROnPlateau()])


In [ ]:
inception_model.summary()

In [ ]:
evaluate_model(inception_model, val_generator, "InceptionV3")

# Save the models

In [ ]:
# Save the trained models
vgg_model.save("vgg16_plant_disease.h5")
resnet_model.save("resnet50_plant_disease.h5")
inception_model.save("inceptionv3_plant_disease.h5")
print("Models Saved Successfully!")


Best Model

In [ ]:
from tensorflow.keras.models import load_model

# Load the best model (change file name if using VGG16 or ResNet50)
best_model = load_model("inceptionv3_plant_disease.h5")

# Load class names
classes = [
    "Pepper__bell___Bacterial_spot",
    "Pepper__bell___healthy",
    "Potato___Early_blight",
    "Potato___healthy",
    "Potato___Late_blight",
    "Tomato_Bacterial_spot",
    "Tomato_Early_blight",
    "Tomato_healthy",
    "Tomato_Late_blight",
    "Tomato_Leaf_Mold",
    "Tomato_Septoria_leaf_spot",
    "Tomato_Spider_mites_Two_spotted_spider_mite",
    "Tomato__Target_Spot",
    "Tomato__Tomato_mosaic_virus",
    "Tomato__Tomato_YellowLeaf__Curl_Virus",
]


In [ ]:
# Dictionary of precautionary measures
precautions = {
    "Bacterial Spot": "Remove infected plants, avoid overhead watering, use copper-based fungicides.",
    "Early Blight": "Use disease-resistant varieties, rotate crops, remove affected leaves.",
    "Late Blight": "Apply fungicides, avoid wet conditions, destroy infected plants.",
    "Leaf Mold": "Ensure good air circulation, reduce humidity, use fungicides.",
    "Septoria Leaf Spot": "Use fungicides, remove infected leaves, avoid overhead watering.",
    "Spider Mites": "Spray with neem oil, maintain humidity, introduce beneficial insects.",
    "Target Spot": "Use fungicides, rotate crops, keep the garden clean.",
    "Tomato Mosaic Virus": "Remove infected plants, disinfect tools, control aphids.",
    "Tomato Yellow Leaf Curl Virus": "Control whiteflies, remove infected plants, use virus-resistant varieties.",
}


# Gradio UI

In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing import image

# Function to Predict Disease & Suggest Solution
def predict_plant_disease(img):
    img = img.resize((150, 150))  # Resize image to match model input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimension for batch
    img_array /= 255.0  # Normalize pixel values

    # Predict using the model
    predictions = best_model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class = classes[predicted_class_index]

    # Get Precautionary Advice
    disease_name = predicted_class.split("___")[-1]  # Extract disease type
    precaution = precautions.get(disease_name, "No specific precautions available.")

    return f"Predicted Class: {predicted_class}", f"Precaution: {precaution}"

# Build Gradio UI
gr.Interface(
    fn=predict_plant_disease,
    inputs=gr.Image(type="pil"),
    outputs=["text", "text"],
    title="Plant Disease Detection",
    description="Upload a leaf image, and the model will predict the disease & suggest precautions.",
).launch()
